<a href="https://colab.research.google.com/github/Katanov/DA_projects/blob/main/TestAeroflot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Загрузка данных и предобработка

In [97]:
import pandas as pd
import numpy as np

In [98]:
import sqlite3

In [99]:
sqlite3.sqlite_version

'3.36.0'

In [100]:
#!gdown --id 1BSHIKQ7rFw5BpTq5nw1UZfjPK_7Mpnbi
#!mv _sqlite3.cpython-37m-x86_64-linux-gnu.so /usr/lib/python3.7/lib-dynload/
#import os
#os.kill(os.getpid(), 9)

In [101]:
con = sqlite3.connect('db')

In [102]:
df = pd.read_csv('/content/Новая таблица - Лист1 (2).csv')
df1 = pd.read_csv('/content/Новая таблица - Лист2 (8).csv')
df2 = pd.read_csv('/content/Новая таблица - Лист3 (2).csv')

In [103]:
df1['txn_date'] = pd.to_datetime(df1['txn_date'],format='%d.%m.%Y')

In [104]:
df.head(2)

,client_id,client_number
0,1,Client number 1
1,2,Client number 2


In [105]:
df1.tail(5)

,client_id,txn_ID,txn_date,txn_type,txn_sts,product_id,miles_sum,Client_mile_balance_after_txn
28,999,Txn ID 29,2019-08-26,Accrual,Processed,Product type id 2,120000,55555
29,999,Txn ID 30,2019-08-27,Redemption,Cancelled,Product type id 2,120000,55555
30,999,Txn ID 31,2020-05-25,Redemption,Cancelled,Product type id 1,120000,55555
31,999,Txn ID 32,2020-05-26,Redemption,Processed,Product type id 1,120000,55555
32,999,Txn ID 33,2020-05-27,Redemption,Processed,Product type id 1,120000,55555


In [106]:
df2

,product_id,product_nm
0,Product type id 1,Ticket
1,Product type id 2,Bank card
2,Product type id 3,Hotel


In [107]:
df.to_sql('clients',con,index=False,if_exists='replace')
df1.to_sql('trans',con,index=False,if_exists='replace')
df2.to_sql('product',con,index=False,if_exists='replace')

In [108]:
def select(sql):
  return pd.read_sql(sql,con)

# Решение и комментарии

Создал небольшой датасет на основе структуры таблиц, ибо, к сожалению, не могу писать запросы без возможности потрогать данные и проследить логику запроса в действии.

Касательно оформления:
- не всегда использую псевдонимы, ибо в большинстве случаев это избыточно и почти не влияет на читаемость.
- постоянно использую CTE, так как учился на пандасе и там всегда было проще работать пошагово через создание большого количества таблиц, а не пытаться уместить все действия сразу в один запрос.

Касательно запроса:
- скорее всего нужно было еще сделать group by по номерам транзакций и определить для каждой транзакции последнее событие, но опять же: так как не было самих данных, то здесь можно придумать разную логику, поэтому я опустил этот шаг. 

In [109]:
sql = '''
with 

trans_with_product as(
select * 
from trans t
left join product p on t.product_id = p.product_id
), 
-- джойним основную таблицу с таблицей с продуктами

rnk as(
select t.*,
row_number() over (partition by client_id order by txn_date) as rnk
from trans_with_product t), 
-- нумеруем транзакции клиентов по дате

min_rnk as(
select client_id, min(rnk.rnk) as rnk
from rnk
where product_nm = 'Ticket'
and txn_sts = 'Processed'
and date(txn_date, 'start of year') = '2020-01-01'
group by client_id), 
-- определяем rnk транзакции первого полета в 2020

miles_before as(
select client_id, rnk-1 as miles_before 
from min_rnk), 
-- определяем количество миль на счету перед первым полетом в 2020

final_almost as(
select rnk.* 
from rnk, miles_before
where rnk.client_id = miles_before.client_id
and rnk.rnk = miles_before.miles_before
and Client_mile_balance_after_txn >= 40000), 
-- фильтруем клиентов по количеству миль перед первым полетом в 2020

more50k as(
select client_id, sum(miles_sum) as sum2020, count(*) as cnt, avg(miles_sum) as avg
from rnk
where txn_type = 'Redemption'
and txn_sts = 'Processed'
and date(txn_date, 'start of year') = '2020-01-01'
group by client_id), 
-- определяем клиентов, потративших свыше 50к миль на полеты

final as(
select f.client_id, cnt as количество_транзакций, avg as среднее_количество_миль
from more50k m, final_almost f
where m.client_id = f.client_id)

select c.client_number as номер_клиента, f.количество_транзакций, f.среднее_количество_миль 
from final f
left join clients c on f.client_id = c.client_id
group by f.client_id
'''

In [110]:
select(sql)

,номер_клиента,количество_транзакций,среднее_количество_миль
0,Client number 28,2,120000.0
